In [2]:
"""Preprocessing for news features."""

import os
import pandas as pd
import numpy as np
import nltk
import re

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sentence_transformers import SentenceTransformer

# # Baixar recursos do NLTK
# nltk.download("stopwords")
# from nltk.corpus import stopwords

os.chdir("c:/Users/gufer/OneDrive/Documentos/FIAP/Fase_05/ML_Engineer_Datathon/src/features")

# pd.set_option('display.max_colwidth', None) 
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import pandas as pd
import re
from utils import concatenate_csv_to_df
from constants import (
    NEWS_TEMP_PATH, 
    NEWS_N_CSV_FILES,
    cols_to_clean,
    cols_to_drop)

os.chdir("c:/Users/gufer/OneDrive/Documentos/FIAP/Fase_05/ML_Engineer_Datathon/")

In [20]:
def pre_process_news() -> pd.DataFrame:
    
    # Concatena CSVs
    df_news = concatenate_csv_to_df(NEWS_TEMP_PATH, NEWS_N_CSV_FILES)
    df_news["page"].rename("pageId", inplace=True)
    
    # Converte para datetimes nas colunas de data
    df_news['issued'] = pd.to_datetime(df_news['issued'])
    df_news['modified'] = pd.to_datetime(df_news['modified'])

    # Criar colunas separadas para data e hora
    df_news['issuedDate'] = df_news['issued'].dt.date
    df_news['issuedTime'] = df_news['issued'].dt.time
    df_news['modifiedDate'] = df_news['modified'].dt.date
    df_news['modifiedTime'] = df_news['modified'].dt.time
 
    # Extrai o miolo relevante da URL
    regex = r'(?<=g1\.globo\.com\/)(.*?)(?=\/noticia)'
    df_news['urlExtracted'] = df_news['url'].str.extract(regex)
    
    # Função para identificar localidades
    def check_local(url):
        if pd.notna(url):
            # Padrão para localidades: duas letras, barra, nome da localidade
            regex = re.compile(r'^[a-z]{2}/[a-z-]+')
            match = regex.match(url)
            if match:
                # Retorna apenas a parte da localidade
                return match.group()  
        return None

    # Função para identificar temas
    def check_theme(url):
        localidade = check_local(url)
        if pd.notna(url):
            if localidade:
                # Se houver uma localidade, o tema é o restante da URL após a localidade
                tema = url.replace(localidade, '').lstrip('/')
                return tema if tema else None
            else:
                # Se não houver localidade, a URL inteira é o tema
                return url
        return None

    # Aplicar as funções ao DataFrame
    df_news['local'] = df_news['urlExtracted'].apply(check_local)
    df_news['localState'] = df_news['local'].str.split('/').str[0]
    df_news['localRegion'] = df_news['local'].str.split('/').str[1]
    df_news['theme'] = df_news['urlExtracted'].apply(check_theme)
    df_news['themeMain'] = df_news['theme'].str.split('/').str[0]
    df_news['themeSub'] = df_news['theme'].str.split('/').str[1]
    
    cols_to_clean = ["body", "title", "caption", "urlExtracted"]
    cols_to_drop = ["local", "theme", "issued", "modified", "url"] + cols_to_clean
    
    def preprocess_text(text):
        text = re.sub(r'\W+', ' ', text)  # Remove caracteres especiais
        text = re.sub(r'\d+', '', text)  # Remove números
        text = text.lower()  # Converte para minúsculas
        return text

    for col in cols_to_clean:
        df_news[f"{col}Cleaned"] = df_news[col].fillna("").astype(str).apply(preprocess_text)

    # Dropa colunas após limpeza
    df_news.drop(columns=cols_to_drop, inplace=True)
    
    return df_news

df_news = pre_process_news()

In [22]:
df_news.columns

Index(['page', 'issuedDate', 'issuedTime', 'modifiedDate', 'modifiedTime',
       'urlExtracted', 'localState', 'localRegion', 'themeMain', 'themeSub',
       'bodyCleaned', 'titleCleaned', 'captionCleaned'],
      dtype='object')

In [ ]:
"""Preprocessing for news features."""



def preprocess_news() -> pd.DataFrame:
    """
    Realiza o pré-processamento dos dados de notícias:
    - Concatena CSVs.
    - Extrai informações da URL (localidade, tema da notícia).
    - Limpa colunas de texto.
    - Remove colunas desnecessárias.
    """
    # Concatena CSVs
    df_news = concatenate_csv_to_df(NEWS_TEMP_PATH, NEWS_N_CSV_FILES)

    # Renomeia coluna de chave primária
    df_news = df_news.rename(columns={"page": "pageId"})

    # Converte colunas de data de publicação e modificação
    for col in ['issued', 'modified']:
        df_news[col] = pd.to_datetime(df_news[col])
        df_news[f'{col}Date'] = df_news[col].dt.date
        df_news[f'{col}Time'] = df_news[col].dt.time

    # Extrai informações do miolo da URL
    df_news['urlExtracted'] = df_news['url'].apply(_extract_url_midsection)

    # Extrai localidade da URL
    df_news['local'] = df_news['urlExtracted'].apply(_extract_location)
    df_news['localState'] = df_news['local'].str.split('/').str[0]
    df_news['localRegion'] = df_news['local'].str.split('/').str[1]

    # Extrai tema da notícia da URL
    df_news['theme'] = df_news['urlExtracted'].apply(lambda x: _extract_theme(x, _get_local_for_url(df_news, x)))
    df_news['themeMain'] = df_news['theme'].str.split('/').str[0]
    df_news['themeSub'] = df_news['theme'].str.split('/').str[1]

    # Limpa colunas de texto
    for col in cols_to_clean:
        df_news[f"{col}Cleaned"] = df_news[col].apply(_preprocess_text)

    # Remove colunas desnecessárias
    df_news = df_news.drop(columns=cols_to_drop)

    return df_news

def _extract_url_midsection(url):
    """Extrai o miolo relevante da URL."""
    regex = r'(?<=g1\.globo\.com\/)(.*?)(?=\/noticia)'
    match = re.search(regex, url)
    return match.group() if match else None

def _extract_location(url_part):
    """Extrai a localidade a partir do miolo da URL."""
    if not url_part:
        return None
    regex = re.compile(r'^[a-z]{2}/[a-z-]+')
    match = regex.match(url_part)
    return match.group() if match else None

def _extract_theme(url_part, location):
    """Extrai o tema da notícia a partir do miolo da URL."""
    if not url_part:
        return None
    if location:
        theme = url_part.replace(location, '').lstrip('/')
        return theme if theme else None
    return url_part

def _preprocess_text(text):
    """Padroniza e limpa o texto de notícias."""
    if not isinstance(text, str):
        text = "" # Trata valores não string como vazios
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\d+', '', text)
    return text.lower()

def _get_local_for_url(df, url_extracted):
    try:
        return df.loc[df['urlExtracted'] == url_extracted, 'local'].iloc[0]
    except (IndexError, KeyError):
        return None
    
preprocess_news()

In [ ]:
### **1. TF-IDF Vectori'zation**

# Lista de stopwords em português
stopwords_pt = set(stopwords.words("portuguese"))

# Tokenização para modelos temáticos (LDA)
texts_tokenized = [
    [word for word in text.split() if word not in stopwords_pt] for text in texts_clean
]
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=list(stopwords_pt))
tfidf_matrix = tfidf_vectorizer.fit_transform(texts_clean).toarray()

In [12]:
### **2. Word Embeddings com Sentence Transformers (BERT-like)**
# embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # Modelo leve e eficiente
# embeddings = embedder.encode(texts_clean.to_list(), convert_to_numpy=True)

KeyboardInterrupt: 